In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from tqdm import tqdm_notebook

import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

%matplotlib inline


from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [323]:
##links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [431]:
def change_string(s):
    return ' '.join(s.lower().replace(' ', '').replace('-', '').split('|'))

In [432]:
# Нормализуем жанры
movies['genres_formatted'] = movies.genres.apply(change_string)


In [433]:
movies=movies.drop(['genres'], axis=1)

In [324]:
# слепим теги каждого юзера,  если они относятся к одному фильму
tags['tag_new'] = tags[['userId','movieId','tag']].groupby(['userId','movieId'])['tag'].transform(lambda x: ' '.join(x))
tags=tags[['userId','movieId','tag_new']].drop_duplicates()

### Причешем наши теги

In [11]:
import pymorphy2
from nltk.tokenize import word_tokenize
import nltk
#nltk.download('punkt')
from tqdm import tqdm

In [12]:
class MorphProvider:
    def __init__(self):
        self.cache = {}
        self.morph = pymorphy2.MorphAnalyzer()
    
    def __call__(self, w):
        w = w.lower()
        cached = self.cache.get(w)
        if cached:
            return cached
        try:
            morphed = self.morph.parse(w)[0].normal_form
            self.cache[w] = morphed
            return morphed
        except:
            return None
    
    def morph_string(self, s):
        words = word_tokenize(s)
        return " ".join([self.__call__(w) for w in words])

In [13]:
morph = MorphProvider()

In [327]:
tags["tag_normalized"] = [morph.morph_string(t) for t in tqdm(tags.tag_new)]

100%|███████████████████████████████████████████████████████████████████████████| 1775/1775 [00:00<00:00, 10320.87it/s]


In [328]:
tags=tags.drop(['tag_new'], axis=1)

In [329]:
tags.head()

,userId,movieId,tag_normalized
0,2,60756,funny highly quotable will ferrell
3,2,89774,boxing story mma tom hardy
6,2,106782,drugs leonardo dicaprio martin scorsese
9,7,48516,way too long
10,18,431,al pacino gangster mafia


## Сделаем какой-то бейслайн простой:

### Посчитаем разные метрики

In [51]:
raitis_aggr= ratings.groupby(['movieId'])[['rating']].count()
raitis_aggr.columns=['count_']
raitis_aggr['mean_']= ratings.groupby(['movieId'])[['rating']].mean()
raitis_aggr['median_']= ratings.groupby(['movieId'])[['rating']].median()
raitis_aggr['var_']= ratings.groupby(['movieId'])[['rating']].var()
raitis_aggr['mode']=ratings.groupby(['movieId']).rating.agg(lambda x: stats.mode(x)[0])

In [73]:
raitis_aggr.shape

(9724, 5)

In [408]:
raitis_aggr.head(4)

,count_,mean_,median_,var_,mode
movieId,,,,,
1,215,3.920930,4.0,0.696990,4.0
2,110,3.431818,3.5,0.777419,4.0
3,52,3.259615,3.0,1.112651,3.0
4,7,2.357143,3.0,0.726190,3.0


In [409]:
ratings[['userId','movieId','rating']].shape

(100836, 3)

In [410]:
x = pd.merge(ratings[['userId','movieId','rating']], raitis_aggr, on='movieId', how='left')

In [411]:
y=x.rating


In [412]:
x=x.drop(['rating'], axis=1)
x=x.drop(['userId'], axis=1)
x=x.drop(['movieId'], axis=1)

In [413]:
x.head()

,count_,mean_,median_,var_,mode
0,215,3.920930,4.0,0.696990,4.0
1,52,3.259615,3.0,1.112651,3.0
2,102,3.946078,4.0,0.667856,4.0
3,203,3.975369,4.0,0.850875,4.0
4,204,4.237745,4.5,0.641475,5.0


In [414]:

x.isnull().any()

count_     False
mean_      False
median_    False
var_        True
mode       False
dtype: bool

In [417]:
#заменим  пропуски в дисперсии на ноль
x.var_  = x.var_.fillna(0)


In [418]:
x_means=x.copy()

### Обучим на этих признаках какой-нибдуь регрессор

In [420]:

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=42)

In [421]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

C:\Users\gpog001\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [422]:


predictions = model.predict(X_test)

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, predictions)))

root_mean_squared_error =  0.9324357392932477


In [213]:
print(model.feature_importances_)

[0.03125655 0.88192114 0.01191224 0.06062738 0.0142827 ]


## Попробуем на жанрах обучиться

In [428]:
x = ratings[['userId','movieId','rating']]

In [435]:
movies.head(2)

,movieId,title,genres_formatted
0,1,Toy Story (1995),adventure animation children comedy fantasy
1,2,Jumanji (1995),adventure children fantasy


In [436]:
x = pd.merge(x, movies[['movieId','genres_formatted']], on='movieId', how='left')

In [437]:
x.shape

(100836, 4)

In [438]:
x.head()

,userId,movieId,rating,genres_formatted
0,1,1,4.0,adventure animation children comedy fantasy
1,1,3,4.0,comedy romance
2,1,6,4.0,action crime thriller
3,1,47,5.0,mystery thriller
4,1,50,5.0,crime mystery thriller


In [439]:

x.isnull().any()

userId              False
movieId             False
rating              False
genres_formatted    False
dtype: bool

In [440]:
x=x.drop(['rating'], axis=1)
x=x.drop(['userId'], axis=1)
x=x.drop(['movieId'], axis=1)

In [441]:
cv_genres = CountVectorizer()
tf_genres = TfidfTransformer()

In [442]:
cv_genres = cv_genres.fit_transform(x.genres_formatted)

In [443]:
tfidf_genres= tf_genres.fit_transform(cv_genres)

In [444]:
tfidf_genres

<100836x20 sparse matrix of type '<class 'numpy.float64'>'
	with 274480 stored elements in Compressed Sparse Row format>

In [476]:

X_train, X_test, y_train, y_test = train_test_split(tfidf_genres, y, test_size=0.3,random_state=42)

In [477]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

C:\Users\gpog001\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [478]:


predictions = model.predict(X_test)

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, predictions)))

root_mean_squared_error =  1.0054298633952983


In [448]:
print(model.feature_importances_)

[0.10132511 0.03864482 0.020373   0.08034378 0.1227395  0.08492534
 0.00400695 0.18255028 0.04780026 0.00406611 0.05621374 0.0123576
 0.0167879  0.03467978 0.00070085 0.04079445 0.03894128 0.04954908
 0.04569616 0.017504  ]


## Попробуем на тегах обучиться

In [449]:
tags.head()

,userId,movieId,tag_normalized
0,2,60756,funny highly quotable will ferrell
3,2,89774,boxing story mma tom hardy
6,2,106782,drugs leonardo dicaprio martin scorsese
9,7,48516,way too long
10,18,431,al pacino gangster mafia


In [450]:
x.head()

,genres_formatted
0,adventure animation children comedy fantasy
1,comedy romance
2,action crime thriller
3,mystery thriller
4,crime mystery thriller


In [451]:
x = ratings[['userId','movieId','rating']]

In [452]:
tags.head()

,userId,movieId,tag_normalized
0,2,60756,funny highly quotable will ferrell
3,2,89774,boxing story mma tom hardy
6,2,106782,drugs leonardo dicaprio martin scorsese
9,7,48516,way too long
10,18,431,al pacino gangster mafia


In [453]:
tags.shape

(1775, 3)

In [455]:
x = pd.merge(x, tags[['movieId','userId','tag_normalized']], on=['userId','movieId'], how='left')

In [456]:
x.shape

(100836, 4)

In [458]:

x.isnull().any()

userId            False
movieId           False
rating            False
tag_normalized     True
dtype: bool

In [459]:
x.tag_normalized = x.tag_normalized.fillna('')

In [460]:
x=x.drop(['rating'], axis=1)
x=x.drop(['userId'], axis=1)
x=x.drop(['movieId'], axis=1)

In [461]:
x.head()

,tag_normalized
0,
1,
2,
3,
4,


In [462]:
x.shape

(100836, 1)

In [463]:
cv_tags = CountVectorizer()
tf_tags = TfidfTransformer()

In [464]:
cv_tags = cv_tags.fit_transform(x.tag_normalized)

In [465]:
tfidf_tags= tf_tags.fit_transform(cv_genres)

In [466]:
tfidf_tags

<100836x20 sparse matrix of type '<class 'numpy.float64'>'
	with 274480 stored elements in Compressed Sparse Row format>

In [479]:

X_train, X_test, y_train, y_test = train_test_split(tfidf_tags, y, test_size=0.3,random_state=42)

In [480]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

C:\Users\gpog001\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [481]:


predictions = model.predict(X_test)

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, predictions)))

root_mean_squared_error =  1.0061095975281757


In [470]:
print(model.feature_importances_)

[0.09436135 0.05091047 0.02142314 0.07997128 0.13804687 0.07374758
 0.00582191 0.17535037 0.0457826  0.00751533 0.05757872 0.01512987
 0.01672112 0.034148   0.00028671 0.03819048 0.04044447 0.04904691
 0.04033243 0.01519037]


## Гм, а если на тех и на других призанках?

In [482]:
X_train, X_test, y_train, y_test = train_test_split(np.hstack((tfidf_tags.toarray(),tfidf_genres.toarray())), y, test_size=0.3,random_state=42)

In [483]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

C:\Users\gpog001\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [484]:


predictions = model.predict(X_test)

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, predictions)))

root_mean_squared_error =  1.0060322061109381


In [485]:
print(model.feature_importances_)

[0.04398855 0.023545   0.01092936 0.04237197 0.04717207 0.03690032
 0.00214453 0.08301362 0.02119354 0.00282084 0.03872261 0.00716818
 0.00720475 0.01529311 0.00039507 0.01847076 0.02615267 0.02818758
 0.01601683 0.00638265 0.04844721 0.02799451 0.0125182  0.034204
 0.08218842 0.04490486 0.00262153 0.08771518 0.02282666 0.00644977
 0.02060861 0.00617644 0.00564571 0.02638434 0.00028072 0.01912532
 0.02209199 0.02260768 0.02144678 0.0076881 ]


In [486]:
x_means.values.shape

(100836, 5)

### Добавим еще наши средние

In [487]:
X_train, X_test, y_train, y_test = train_test_split(np.hstack((tfidf_tags.toarray(),tfidf_genres.toarray(),x_means.values)), y, test_size=0.3,random_state=42)

In [488]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

C:\Users\gpog001\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [489]:


predictions = model.predict(X_test)

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, predictions)))

root_mean_squared_error =  0.9461828049174449


In [490]:
print(model.feature_importances_)

[3.73446389e-03 2.94359705e-03 9.07349118e-04 1.70844537e-03
 4.96832764e-03 2.37141610e-03 6.68651856e-04 5.32524275e-03
 2.17888560e-03 2.20546370e-04 2.34509955e-03 6.11401985e-04
 1.18144331e-03 1.67744595e-03 5.07870713e-05 3.25848302e-03
 1.87892901e-03 3.00424960e-03 1.18029649e-03 6.09103471e-04
 3.92574318e-03 2.76184255e-03 1.40891245e-03 1.47082284e-03
 5.61385466e-03 2.19703674e-03 5.45419302e-04 4.88535054e-03
 1.97798626e-03 2.61828615e-04 2.24774179e-03 6.94289716e-04
 1.00952263e-03 1.63864190e-03 3.25641814e-05 3.37829281e-03
 2.51675703e-03 3.27085017e-03 1.14884691e-03 7.34733069e-04
 1.92260179e-02 8.41528276e-01 9.08480282e-03 3.75312010e-02
 1.00844996e-02]


###  И так мы тоже не превзоошли бейс-лайн